# An Analysis of Political Contributions During the 2020 House of Representatives Election

In this part, you will obtain as much data as you can on the campaign contributions received by each candidate. This data is avaiable through the website https://www.opensecrets.org/.

### Part 1: Data Gathering

#### 1. Start by acquiring the data from Tennessee's 7th District, which is available at https://www.opensecrets.org/races/summary?cycle=2020&id=TN07&spec=N. If you click the "Download .csv file", you can get a csv for this district. However, we don't want to have to click this button across all districts. Instead, we'll use Python to help automate this process. Start by sending a get request to the download button URL, https://www.opensecrets.org/races/summary.csv?cycle=2020&id=TN07. Convert the result to a DataFrame.

In [22]:
import requests
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime as dt
import urllib3
# import re2
import regex
import re
import io
from requests.exceptions import HTTPError

In [23]:
url = 'https://www.opensecrets.org/races/summary.csv?cycle=2020&id=TN07'

#Lets use try-except whenever we make any any http request.

#If we invoke .raise_for_status(), then Requests will raise an HTTPError for status codes between 400 and 600. 
#If the status code indicates a successful request, then the program will proceed without raising that exception.

try:
    response = requests.get(url)
    response.raise_for_status()
except HTTPError as http_err:
    print(f"HTTP error occurred: {http_err}")
except Exception as err:
    print(f"Other error occurred: {err}")
else:
    data = response.content.decode('utf8')
    df = pd.read_csv(io.StringIO(data))

#Lets populate the district ID column with TN07 so that we could use it later
df['DistIDCurr']='TN07'
df


,cid,FirstLastP,Rcpts,Spent,PACs,Indivs,Cand,Other,EndCash,LgIndivs,...,Result,CRPICO,State,IncCID,Incumbent,primarydate,DistIDCurr,capeye,sort,SmLgIndivsNote
0,N00041873,Mark Green (R),1194960.47,935486.67,171900.0,819151.42,0.0,203909.05,287888.55,819151.42,...,W,I,Tennessee,NaN,NaN,2020-08-06 00:00:00 +0000,TN07,0,1,N
1,N00045536,Kiran Sreepada (D),206644.28,207190.98,4000.0,202644.28,0.0,0.00,0.00,179129.75,...,L,C,Tennessee,NaN,NaN,2020-08-06 00:00:00 +0000,TN07,0,2,N
2,N00047077,Ronald Brown (I),1750.00,0.00,0.0,1750.00,0.0,0.00,9006.00,300.00,...,L,C,Tennessee,NaN,NaN,2020-08-06 00:00:00 +0000,TN07,0,2,N
3,N00046592,Scott Vieira Jr (I),655.47,1048.51,10.0,45.00,35.0,565.47,-196.52,0.00,...,L,C,Tennessee,NaN,NaN,2020-08-06 00:00:00 +0000,TN07,0,2,N
4,N00045535,Benjamin Estes (3),0.00,0.00,0.0,0.00,0.0,0.00,0.00,0.00,...,,C,Tennessee,NaN,NaN,2020-08-06 00:00:00 +0000,TN07,0,2,N


#### 2. Once you have working code for Tennessee's 7th District, expand on your code to capture all of Tennessee's districts into a single DataFrame. Make sure that you can distinguish which district each result came from. Export the results to a csv file.


In [42]:
#Lets define function to get the dataFrame for district

def getDistrictData(districtCode):
    url = f'https://www.opensecrets.org/races/summary.csv?cycle=2020&id={districtCode}'

    #Lets use try-except whenever we make any any http request.
    #If we invoke .raise_for_status(), then Requests will raise an HTTPError for status codes between 400 and 600. 
    #If the status code indicates a successful request, then the program will proceed without raising that exception.

    try:
        response = requests.get(url)
        response.raise_for_status()
    except HTTPError as http_err:
        print(f"HTTP error occurred: {http_err}")
    except Exception as err:
        print(f"Other error occurred: {err}")
    else:
        data = response.content.decode('utf8')
        df = pd.read_csv(io.StringIO(data))

    #Lets populate the district ID column with TN07 so that we could use it later
    df['DistIDCurr']= districtCode
    return df

    

#Lets define dictionary with state and district code.
districtList = ['01','02','03','04','05','06','07','08','09']
StateDistricts = {
    "state":'TN',
    "Districts": districtList
}

#lets define the dataframe list which we can use later to cooncatenate all district data
frameList=[]

for i in range(0,len(StateDistricts['Districts'])):
    DistrictCode = f'{StateDistricts["state"]}{StateDistricts["Districts"][i]}'
    frameList.append(getDistrictData(DistrictCode))

#dist08 = getDistrictData('TN08')
#dist08

StateDistrictframe = pd.concat(frameList)
StateDistrictframe



,cid,FirstLastP,Rcpts,Spent,PACs,Indivs,Cand,Other,EndCash,LgIndivs,...,Result,CRPICO,State,IncCID,Incumbent,primarydate,DistIDCurr,capeye,sort,SmLgIndivsNote
0,N00046688,Diana Harshbarger (R),2126945.60,1869099.77,222800.00,359728.50,1461293.00,83124.10,257845.83,315489.10,...,W,O,Tennessee,NaN,NaN,2020-08-06 00:00:00 +0000,TN01,0,2,N
1,N00046686,Blair Nicole Walsingham (D),140209.14,134994.55,1520.00,138689.14,0.00,0.00,5214.59,70085.20,...,L,O,Tennessee,NaN,NaN,2020-08-06 00:00:00 +0000,TN01,0,2,N
2,N00047760,Steve Holder (I),0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,,O,Tennessee,NaN,NaN,2020-08-06 00:00:00 +0000,TN01,0,2,N
0,N00041594,Tim Burchett (R),1336275.75,878487.63,269535.00,1072845.61,0.00,-6104.86,593677.72,729831.26,...,W,I,Tennessee,NaN,NaN,2020-08-06 00:00:00 +0000,TN02,0,1,N
1,N00041699,Renee Hoyos (D),812783.86,816793.15,3100.00,807459.01,0.00,2224.85,209.82,807459.01,...,L,C,Tennessee,NaN,NaN,2020-08-06 00:00:00 +0000,TN02,0,2,N
2,N00047761,Matthew Campbell (I),0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,,C,Tennessee,NaN,NaN,2020-08-06 00:00:00 +0000,TN02,0,2,N
0,N00030815,Chuck Fleischmann (R),1051653.39,381411.20,453858.46,603344.93,0.00,-5550.00,1880341.32,599059.93,...,W,I,Tennessee,NaN,NaN,2020-08-06 00:00:00 +0000,TN03,0,1,N
1,N00046911,Meg Gorman (D),85843.21,77759.83,2671.60,81271.61,2000.00,-100.00,8083.38,50245.20,...,L,C,Tennessee,NaN,NaN,2020-08-06 00:00:00 +0000,TN03,0,2,N
2,N00046589,Nancy Baxley (I),0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,,C,Tennessee,NaN,NaN,2020-08-06 00:00:00 +0000,TN03,0,2,N
3,N00047762,Amber Hysell (I),0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,,C,Tennessee,NaN,NaN,2020-08-06 00:00:00 +0000,TN03,0,2,N
